<a href="https://colab.research.google.com/github/hogo56/BertQA/blob/master/BERTjoint_yes_no_Howard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERTjoint Question Answering Contest

I prefer to do my primary development in a Colab virtual machine but, competitions require your kernel to run on Kaggle for scoring. You can start with this notebook and add your own project code which should run in either location if you use the directory variables for file locations and correctly configure data and user libraries in Kaggle.<p>
The main differences are the file locations. On Colab you will symlink to a google Drive and for Kaggle you will need to upload your files as a .zip data file If you want SSH access there is code at bottom to allow this for Colab (not possible on Kaggle).<p>
If you run into problems or have suggestions I'd love to hear from you.

## Explanation

### Data & Directories
There are a couple of differences between Kaggle and Colab.<p>
**Colab** - Is a live linux system with nothing being persistant. You can attach a google drive to your kernel and/or download files from gs://, GitHub, Kaggle, etc. Using drive can have a performance penalty but is the easiest way to access persistent files.<br>
I am symlinking Library files and output files.<br>
Because my google drive space is limited I choose to download large data files each time the kernel is used.<p>
**Kaggle** - The kernel has a persistent ./input directory for data that is read only. You can attach data and library files there. (zip your files into a single file and upload the .zip).<br>
Your personal ./lib directory can ba zipped and uploaded there as ./input/lib (but I am uncertain if the competition scoring allows use of private data sets.)<br>
You can also create notebooks of kernel type "script" and then file->Add Utility Script in your competition notebook to attach the script files under ./usr/lib but you have to do this one file at a time. (see: https://www.kaggle.com/product-feedback/91185 for more information)<br>
I don't think competition scoring allows internet access so all files have to be attached to your notebook at submit time.<p>
Because I am using symlinks to lib files on google drive if you ssh into the kernel and vim the library files you are changing the persistent copy of the files.

### Switching between Colab & Kaggle
One way of moving your script from Colab to Kaggle to run is:<br>
   * delete all cells from your Kaggle competition notebook<br>
   * download the .ipynb from Colab<br>
   * upload it into the blank Kaggle notebook.<br>
   * delete the cells near the bottom of the notebook that need to be deleted when running on Kaggle<br>
   * update any script parameters (eg. verbose)
   * zip your current library files into a lib.zip and upload to your Kaggle dataset file
   * if you have changed ./data files make sure your Kaggle kernel has the current versions of those files

### Drectory Structure (Notebook)

* Kaggle &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <em>(cwd = /kaggle/working/)</em><br>
  {datadir} = /kaggle/input<br>
  {libdir} = /kaggle/input/lib &nbsp; &nbsp; &nbsp; (or /kaggle/usr/lib)<br>
  {outdir} = /kaggle/working<br>
* Colab &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <em>(cwd = /content/)</em><br>
  {datadir} = /content/data<br>
  {libdir} = /content/lib<br>
  {outdir} = /content/output<br>

#### - Required Libraries
   * {libdir}/bert_utils.py
   * {libdir}/modeling.py
   * {libdir}/tokenization.py

#### - Inputs (competition data)
   * {datadir}/tensorflow2-question-answering/ (from https://www.kaggle.com/c/tensorflow2-question-answering/data)

#### - Required Data (additional packages)
   * {datadir}/bert-joint-baseline/ (from prvi
https://www.kaggle.com/prokaj/bert-joint-baseline; contains model and scripts)

#### - Outputs
   * {outdir}/predictions.json
   * {outdir}/submission.csv<br>
   * {outdir}/eval.tf_record<br>
   * {outdir}/.ipynb_checkpoints/<br>

### Drectory Structure (Google Drive)
The following is the file structure for your google drive:<p>

/My Drive/Colab/ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <em>(this directory should be private)</em><br>
/My Drive/Colab/kaggle.json &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <em>(your personal Kaggle auth file)</em><br>
/My Drive/Colab/{projdir}/ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; (folder for a specific project/competition)<br>
/My Drive/Colab/{projdir}/{notebook}/&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;(individual notebook within project)<br>
/My Drive/Colab/{projdir}/{notebook}/notebook.ipynb<br>
/My Drive/Colab/{projdir}/{notebook}/lib/<br>
/My Drive/Colab/{projdir}/{notebook}/output/<br>
/My Drive/{projdir}/ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <em>(optional directory to be shared between teams)</em><br><p>
Using this directory structure and the variables in the config variables section below you can have any number of projects on your google drive and any number of notebook versions in each project. Each notebook can have a unique set of libraries.<p>
There is also a {nbver} which, if not '' is appended to the end of {outdir} and {libdir} offering additional flexibility if you want (if not just set it to ''}


### GitHub
As I learn more about GitHub I will add information here. But, I think you can choose to make repositaries out of either /My Drive/Colab/{projdir}/ or /My Drive/Colab/{projdir}/{notebook}/   <p>
You can then either pull/push to the shared repo directly from the Colab (using ssh, using %%bash commands in notebook or by linking the google Drive to your local machine and doing it from there.<p>
From within the Colab all files are in your google drive from the path /content/gdrive/My\ Drive\Colab\...

### - Notes
Put notes about your Notebook here







### - Credits / Ancestry
If your notebook is a fork or combination of other notebooks here you should provide links so other people can look at where you built your current work from.<p>
This notebook is a fork of [mmmarchetti's notebook](https://www.kaggle.com/mmmarchetti/tensorflow-2-0-bert-yes-no-answers) which was a fork of [prokaj's - bert joint baseline notebook](https://www.kaggle.com/prokaj/bert-joint-baseline-notebook/notebook).<br>
mmmarchetti made some modifications to slightly improve the code and get the YES / NO answers and leave the unknowns blank.

## Notebook Variables

In [0]:
CompSubmission = False                       # Set to True if submitting to Competition

In [0]:
## Turn this on for development to make sure library updates get reimported
#  will reduce performance so don't use for production.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
## Helper Functions   (probably will move to a library eventually)
import os

# Custom Error Handler
class ExecutionStop(Exception):
    def __init__(self, value): self.value=value
    def __str__(self): return(str(self.value))

#  Show list of file sand directories                 (it seems that this is skipping the symlinks)
def list_files(startpath, exclude=["/.config", "/gdrive"]):

    for root, _, files in os.walk(startpath):
        if any([e in root for e in exclude]):
            continue
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f"{subindent}{f}")

def print_flags():
    if verbose:
        print("\nParameters:")
        FLAGS = tf.flags.FLAGS
        for attr, obj in sorted(FLAGS.__flags.items()):
            print(f"{attr.upper()}={obj.value}")
        print("")

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)

In [0]:
## Set file locations    (these variables are not implemented in the FLAGS code yet)
import os, sys
from pathlib import Path

## Config Variables
verbose = True if not CompSubmission else False  # Turn this off to supress some of the "fyi" output
competition = 'tensorflow2-question-answering'
train_file = ''                       # Set this below when you download the training file
test_file = ''                        # Set this below when you download the test file
projdir = 'bertqa'                    # The project directory on Drive for this competition
notebook = 'BERTjoint_yes_no'         # Subdir on Drive for files specific to this notebook
nbver = ''                            # library/output subfolder for this notebook version, or ''
DownloadBigFiles = True               # Files will not download if already on drive

if Path('/content').exists():
    print("Detected running on Colab")
    kernel = 'Colab'
    basedir = "/content"
    libdir = f"{basedir}/lib"
    datadir = f"{basedir}/data"
    outdir = f"{basedir}/output"      # will be symlinked to a user's private gdrive for persistence
elif Path('/kaggle').exists():
    print("Detected running on Kaggle")
    kernel = 'Kaggle'
    basedir = "/kaggle"
    libdir = f"{basedir}/input/lib"   # this has to be in a writable location
    datadir = f"{basedir}/input"      # this may need to be '../input' for scoring
    outdir = f"{basedir}/working"     # this may need to be '.' for scoring
else:
    raise ExecutionStop("Cannot continue without determining file locations")

Detected running on Colab


# ============= Machine Spinup =============

In [0]:
! zdump PST
if verbose:
    list_files(basedir)

PST  Sat Jan  4 00:07:11 2020 PST
content/
    sample_data/
        README.md
        anscombe.json
        california_housing_test.csv
        mnist_train_small.csv
        california_housing_train.csv
        mnist_test.csv


## -- Setup --

### Google Drive

In [0]:
## File link to Google Drive

######
##  WARNING: This is convienent but a rather bad idea from a security point of view. Mounting your
##           Google Drive in this way makes your entire drive accessible read/write and then you
##           are likely to run code from libraries that could be untrustable.
##           Possible alternatives would be to use read only file sharing links or if you want
##           read/write access (to allow file output for example) I would recommend creating a
##           special Google Drive account that only has files related to your Colab work.
######

if kernel == 'Colab':
    from google.colab import drive
    drive.mount(f"{basedir}/gdrive", force_remount=False)   # true to reread drive

    # Create a shorter shared directory name and avoid having to deal with the space in "My Drive"
    if Path(f"{basedir}/{projdir}").is_symlink():
        ! rm "{basedir}/{projdir}"
    ! ln -s "{basedir}/gdrive/My Drive/{projdir}/" "{basedir}/{projdir}"
    if not Path(f"{basedir}/{projdir}").exists():
        raise ExecutionStop("Symlink to shared project directory not found!")

    ## If you do not want to use the lib directoy from your Google Drive set this block False
    if True:
        if Path(libdir).is_symlink():
            ! rm "{libdir}"
        ! ln -s "{basedir}/gdrive/My Drive/Colab/{projdir}/{notebook}/lib{nbver}/" "{libdir}"
        if not Path(libdir).exists():
            raise ExecutionStop("Project libdir directory not found!")

    ## If you do not want output to be written to your Google Drive set this block False
    if True:
        if Path(outdir).is_symlink():
            ! rm "{outdir}"
        ! ln -s "{basedir}/gdrive/My Drive/Colab/{projdir}/{notebook}/output{nbver}/" "{outdir}"
        if not Path(outdir).exists():
            raise ExecutionStop("Project outdir directory not found!")

    if verbose:
        print('\n', basedir)
        ! ls -l "{basedir}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive

 /content
total 12
lrwxrwxrwx 1 root root   32 Jan  4 00:07 bertqa -> '/content/gdrive/My Drive/bertqa/'
drwx------ 4 root root 4096 Jan  4 00:07 gdrive
lrwxrwxrwx 1 root root   59 Jan  4 00:07 lib -> '/content/gdrive/My Drive/Colab/bertqa/BERTjoint_yes_no/lib/'
lrwxrwxrwx 1 root root   62 Jan  4 00:07 output -> '/content/gdrive/My Drive/Colab/bertqa/BERTjoint_yes_no/output/'
drwxr-xr-x 1 root root 4096 Dec 18 16:52 sample_data


### Kaggle API
<Details>You will need Kaggle API token to link the Colab instance to your Kaggle account to get data, etc.<br>
Go to: https://www.kaggle.com/yourID/account and click on the "Create New API Token: button to get a file named kaggle.json.<p>You can put your kaggle.json file in your google drive at My Drive/colab/kaggle.json.<br>
Alternately, you can store it on your local machine and the script will ask you to upload it.</Details>

In [0]:
## Link to Kaggle
if kernel == 'Colab':
    from google.colab import files
    if Path(f"{basedir}/gdrive/My Drive/Colab/kaggle.json").exists():
        # if there is a kaggle.json file in gdrive use it
        os.environ['KAGGLE_CONFIG_DIR'] = f"{basedir}/gdrive/My Drive/Colab/"
        ! ls -l "{basedir}/gdrive/My Drive/Colab/kaggle.json"
        import kaggle
    else:
        # Have user upload file
        print('Upload kaggle.json.')
        # The files.upload() command is failing sporatically with:
        #   TypeError: Cannot read property '_uploadFiles' of undefined (just run this cell again)
        ! rm "{basedir}/kaggle.json"  2> /dev/null
        files.upload()
        ! chmod 600 kaggle.json
        os.environ['KAGGLE_CONFIG_DIR'] = f"{basedir}/"
        ! ls -l "{basedir}/kaggle.json"
        import kaggle


-rw------- 1 root root 66 Dec 15 08:31 '/content/gdrive/My Drive/Colab/kaggle.json'


## -- Main System Config --
<Details><Summary>Global Config</Summary>
Put any global system configuration here

In [0]:
if kernel == "Colab":
    if Path(f"{basedir}/sample_data").exists():
        ! rm -rf "{basedir}/sample_data"

In [0]:
%%bash -s "{libdir}" "{datadir}" "{outdir}"
# make directories if not already exist
[ -d "$1" ] || mkdir -p "$1"        # {libdir}
[ -d "$2" ] || mkdir -p "$2"        # {datadir}
[ -d "$3" ] || mkdir -p "$3"        # {outdir}
zdump PST

PST  Sat Jan  4 00:07:45 2020 PST


In [0]:
import sys, os
if not libdir in sys.path:                       # don't add multiple times
    sys.path.append(libdir)
if not (libdir in os.environ['PYTHONPATH']):     # needed to run python scripts from shell
    os.environ['PYTHONPATH'] += f":{libdir}"

In [0]:
if verbose:
    !pwd
    !ls -l
    print('', "sys.path:", *sys.path, '', sep='\n')
    !printenv |grep -E 'KAGGLE|PYTHON'
    print()
! zdump PST

/content
total 12
lrwxrwxrwx 1 root root   32 Jan  4 00:07 bertqa -> '/content/gdrive/My Drive/bertqa/'
drwxr-xr-x 2 root root 4096 Jan  4 00:07 data
drwx------ 4 root root 4096 Jan  4 00:07 gdrive
lrwxrwxrwx 1 root root   59 Jan  4 00:07 lib -> '/content/gdrive/My Drive/Colab/bertqa/BERTjoint_yes_no/lib/'
lrwxrwxrwx 1 root root   62 Jan  4 00:07 output -> '/content/gdrive/My Drive/Colab/bertqa/BERTjoint_yes_no/output/'

sys.path:

/env/python
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.6/dist-packages/IPython/extensions
/root/.ipython
/content/lib

KAGGLE_CONFIG_DIR=/content/gdrive/My Drive/Colab/
PYTHONWARNINGS=ignore:::pip._internal.cli.base_command
PYTHONPATH=/env/python:/content/lib

PST  Sat Jan  4 00:07:52 2020 PST


# =========== Project Specific Stuff ===========

## -- Project Setup --

### Download Dataset and Support Files

Kaggle Competition Files

In [0]:
## Competition Dataset  (5GB zipped)
if DownloadBigFiles and kernel == 'Colab':
    if not Path(f"{datadir}/compdata.flag").exists():      ## Don't download again if exists
        if verbose:
            ! kaggle competitions list
            print()
        print("Downloading Competition Data\n")
        ! kaggle competitions download -c "{competition}" -p "{datadir}"
        ! mkdir -p "{datadir}/{competition}/"
        ! mv "{datadir}/sample_submission.csv"  "{datadir}/{competition}"
        ! unzip "{datadir}/simplified-nq-train.jsonl.zip" -d "{datadir}/{competition}"
        ! rm "{datadir}/simplified-nq-train.jsonl.zip"
        ! unzip "{datadir}/simplified-nq-test.jsonl.zip" -d "{datadir}/{competition}"
        ! rm "{datadir}/simplified-nq-test.jsonl.zip"
        ! touch "{datadir}/compdata.flag"
    else:
        print("Competition Data already exists. Not downloading.\n")
        !ls -l "{datadir}/{competition}"/*
    train_file = f"{datadir}/{competition}/simplified-nq-train.jsonl"
    test_file = f"{datadir}/{competition}/simplified-nq-test.jsonl"
else:
    print("For Kaggle, make sure you download a copy of the competition data into your kernel")
    ! ls -l "{datadir}/{competition}"/*
    train_file = f"{datadir}/{competition}/simplified-nq-train.jsonl"
    test_file = f"{datadir}/{competition}/simplified-nq-test.jsonl"

    # public_dataset = os.path.getsize(f"{test_file}") < 20_000_000
    # private_dataset = os.path.getsize(f"{test_file}") >= 20_000_000

ref                                            deadline             category             reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
digit-recognizer                               2030-01-01 00:00:00  Getting Started   Knowledge       2240           False  
titanic                                        2030-01-01 00:00:00  Getting Started   Knowledge      16405           False  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started   Knowledge       5384           False  
connectx                                       2030-01-01 00:00:00  Getting Started   Knowledge         14           False  
imagenet-object-localization-challenge         2029-12-31 07:00:00  Research          Knowledge         61           False  
competitive-data-science-predict-future-sales  2020-12-31 23:59:00  Playground            Kudos       5348           False  


BERTjoint files from: https://www.kaggle.com/prokaj/bert-joint-baseline

In [0]:
# Get BERTjoint model files (this a copy of the prokaj file from my Google Drive)
if DownloadBigFiles and kernel == 'Colab':
    if not Path(f"{datadir}/bertfiles.flag").exists():      ## Don't download again if exists
        print("Downloading BERT-joint Model\n")
        ! mkdir -p "{datadir}/bert-joint-baseline/"
        filestoget = "bert_config* model_cpkt* nq-test* vocab*"
        ! kaggle datasets download -d prokaj/bert-joint-baseline -p "{datadir}"
        ! unzip "{datadir}/bert-joint-baseline.zip" {filestoget} -d "{datadir}/bert-joint-baseline/"
        ! rm "{datadir}/bert-joint-baseline.zip"
        if Path(f"{datadir}/bert-joint-baseline-output.npz").exists():
            ! rm "{datadir}/bert-joint-baseline-output.npz" # if kaggle downloaded this delete it
        if verbose:
            ! ls -l "{datadir}/bert-joint-baseline/"
        ! touch "{datadir}/bertfiles.flag"
    else:
        print("BERT-joint Files already exists. Not downloading.\n")
        ! ls -l "{datadir}/bert-joint-baseline/"
else:
    print("For Kaggle, make sure you download a copy of prokaj's bert-joint-baseline to your kernel")
    ! ls -l "{datadir}/bert-joint-baseline/"



 99% 1.52G/1.53G [00:42<00:01, 10.3MB/s]
100% 1.53G/1.53G [00:42<00:00, 38.4MB/s]
Archive:  /content/data/bert-joint-baseline.zip
  inflating: /content/data/bert-joint-baseline/bert_config.json  
  inflating: /content/data/bert-joint-baseline/model_cpkt-1.data-00000-of-00002  
  inflating: /content/data/bert-joint-baseline/model_cpkt-1.data-00001-of-00002  
  inflating: /content/data/bert-joint-baseline/model_cpkt-1.index  
  inflating: /content/data/bert-joint-baseline/nq-test.tfrecords  
  inflating: /content/data/bert-joint-baseline/vocab-nq.txt  
total 1339896
-rw-r--r-- 1 root root        314 Nov 17 15:02 bert_config.json
-rw-r--r-- 1 root root      79677 Nov 17 15:02 model_cpkt-1.data-00000-of-00002
-rw-r--r-- 1 root root 1340596252 Nov 17 15:02 model_cpkt-1.data-00001-of-00002
-rw-r--r-- 1 root root      29452 Nov 17 15:05 model_cpkt-1.index
-rw-r--r-- 1 root root   31094348 Nov 17 15:05 nq-test.tfrecords
-rw-r--r-- 1 root root     231601 Nov 17 15:06 vocab-nq.txt


BERTjoint files from: 
https://github.com/google-research/language/tree/master/language/question_answering/bert_joint


In [0]:
# Get BERTjoint model files    (we are not using this model / files)
if False and DownloadBigFiles and kernel == 'Colab':
    if not Path(f"{datadir}/bertfiles.flag").exists():
        print("Downloading BERT-joint Model\n")
        ! gsutil cp -R gs://bert-nq/bert-joint-baseline "{datadir}""
        ! touch "{datadir}/bertfiles.flag"
    else:
        print("BERT-joint Files already exists. Not downloading.\n")
        ! ls -l "{datadir}/bert-joint-baseline/"
else:
    pass        # if Kaggle, data will already be there

BERT files from: https://github.com/google-research/bert<br>
(Not the model we are using at the moment)

In [0]:
## get BERT (this is unlikely to be the BERT-joint files needed for competition)
# this version of BERT seems won't import as is. On line 88 of lib/bert/optimization.py
#    change   tr.train.Optimizer to tf.keras.optimizers.Optimizer
if False and DownloadBigFiles and kernel == 'Colab':
    ! git clone https://github.com/google-research/bert.git
    ! mv bert lib

    # get some pretrained models  (I really  have no idea what these are or if useful)
    ! wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
    ! unzip cased_L-12_H-768_A-12.zip
    ! rm cased_L-12_H-768_A-12.zip

<Details><Summary>BERT tf.compat.v1 Notes</Summary>
baseline_w_bert_translated_to_tf2_0 and other model libraries being passed around often seem to have tf.compat.v1 in them. Be aware that use of tf.compat.v1, which is not permitted to be eligible for TF2.0 prizes in this competition.<br>
https://www.kaggle.com/dimitreoliveira/using-tf-2-0-w-bert-on-nq-translated-to-tf2-0</Details>

### Library Setup

In [0]:
## Copy lib files from Google Drive   (includes bert-tensorflow & natural-questions)
if kernel == 'Colab':
    ## It is likely that the {libdir} was symlinked to a gDrive directory above
    #  otherwise install code to copy any needed lib files into {libdir}
    pass
if kernel == 'Kaggle':
    ! cp "{datadir}/bert-joint-baseline"/*.py "{libdir}"
if verbose:
    print(f"{libdir}/")
    ! ls -l "{libdir}"/*

/content/lib/
-rw------- 1 root root 44021 Dec 24 20:05 /content/lib/bert_utils.py
-rw------- 1 root root 41596 Dec 24 20:04 /content/lib/modeling.py
-rw------- 1 root root 12411 Dec 24 20:04 /content/lib/tokenization.py

/content/lib/__pycache__:
total 66
-rw------- 1 root root 25440 Dec 25 21:52 bert_utils.cpython-36.pyc
-rw------- 1 root root 31247 Dec 25 21:52 modeling.cpython-36.pyc
-rw------- 1 root root  9861 Dec 25 21:52 tokenization.cpython-36.pyc


In [0]:
## Load Libraries
import os, sys, importlib

if kernel == "Colab":               # Kaggle is V2 by default
    #magic to make Colab path to Tensorflow V2 on Colab
    %tensorflow_version 2.x 

import tensorflow as tf
print("TensofFlow", tf.__version__)

import numpy as np
import pandas as pd
import collections

import bert_utils
import modeling
import tokenization

import json

TensorFlow 2.x selected.
TensofFlow 2.1.0-rc1


In [0]:
## uncomment and use this cell to reimport libs you have updated
# importlibe.reload(bert_utils)

In [0]:
! zdump PST
if verbose:
    list_files(basedir)

PST  Sat Jan  4 00:18:18 2020 PST
content/
    data/
        compdata.flag
        bertfiles.flag
        bert-joint-baseline/
            vocab-nq.txt
            nq-test.tfrecords
            model_cpkt-1.data-00000-of-00002
            bert_config.json
            model_cpkt-1.index
            model_cpkt-1.data-00001-of-00002
        tensorflow2-question-answering/
            simplified-nq-train.jsonl
            simplified-nq-test.jsonl
            sample_submission.csv


In [0]:
# raise ExecutionStop("Execution stopped")

## -- Code Implementation in Tensorflow 2.0 --

**A few notes:**
- If you want to keep using **flags** and **logging** you will have to use the **absl** lib (this is recommended by the TF team).
- Since we won't use it with the kernels, he removed most of the **TPU** related stuff to reduce complexity.
- Tensorflow 2 don't let us use global variables **(tf.compat.v1.trainable_variables())**.

In this notebook, we'll be using the Bert baseline for Tensorflow to create predictions for the Natural Questions test set. Note that this uses a model that has already been pre-trained - we're only doing inference here. A GPU is required, and this should take between 1-2 hours to run.

The original script can be found [here](https://github.com/google-research/language/blob/master/language/question_answering/bert_joint/run_nq.py).<br>
The supporting modules were drawn from the [official Tensorflow model repository](https://github.com/tensorflow/models/tree/master/official).<br>
The bert-joint-baseline data is described [here](https://github.com/google-research/language/tree/master/language/question_answering/bert_joint).

**Note:** This baseline uses code that was migrated from TF1.x. Be aware that it contains use of tf.compat.v1, which is not permitted to be eligible for [TF2.0 prizes in this competition](https://www.kaggle.com/c/tensorflow2-question-answering/overview/prizes).

In [0]:
! zdump PST

PST  Sat Jan  4 00:18:32 2020 PST


### Support Classes

In [0]:
class TDense(tf.keras.layers.Layer):
    def __init__(self,
                 output_size,
                 kernel_initializer=None,
                 bias_initializer="zeros",
                **kwargs):
        super().__init__(**kwargs)
        self.output_size = output_size
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer

    def build(self,input_shape):
        dtype = tf.as_dtype(self.dtype or tf.keras.backend.floatx())
        if not (dtype.is_floating or dtype.is_complex):
          raise TypeError("Unable to build `TDense` layer with "
                          "non-floating point (and non-complex) "
                          "dtype %s" % (dtype,))
        input_shape = tf.TensorShape(input_shape)
        if tf.compat.dimension_value(input_shape[-1]) is None:
          raise ValueError("The last dimension of the inputs to "
                           "`TDense` should be defined. "
                           "Found `None`.")
        last_dim = tf.compat.dimension_value(input_shape[-1])
#       self.input_spec = tf.keras.layers.InputSpec(min_ndim=3, axes={-1: last_dim})    # original value
        self.input_spec = tf.keras.layers.InputSpec(min_ndim=2, axes={-1: last_dim})    # ChrisM changed
        self.kernel = self.add_weight(
            "kernel",
            shape=[self.output_size,last_dim],
            initializer=self.kernel_initializer,
            dtype=self.dtype,
            trainable=True)
        self.bias = self.add_weight(
            "bias",
            shape=[self.output_size],
            initializer=self.bias_initializer,
            dtype=self.dtype,
            trainable=True)
        super(TDense, self).build(input_shape)

    def call(self,x):
        return tf.matmul(x,self.kernel,transpose_b=True)+self.bias


def mk_model(config):
    seq_len = config['max_position_embeddings']
    unique_id  = tf.keras.Input(shape=(1,),dtype=tf.int64,name='unique_id')
    input_ids   = tf.keras.Input(shape=(seq_len,),dtype=tf.int32,name='input_ids')
    input_mask  = tf.keras.Input(shape=(seq_len,),dtype=tf.int32,name='input_mask')
    segment_ids = tf.keras.Input(shape=(seq_len,),dtype=tf.int32,name='segment_ids')
    BERT = modeling.BertModel(config=config,name='bert')
    pooled_output, sequence_output = BERT(input_word_ids=input_ids,
                                          input_mask=input_mask,
                                          input_type_ids=segment_ids)
    
    logits = TDense(2,name='logits')(sequence_output)
    start_logits,end_logits = tf.split(logits,axis=-1,num_or_size_splits= 2,name='split')
    start_logits = tf.squeeze(start_logits,axis=-1,name='start_squeeze')
    end_logits   = tf.squeeze(end_logits,  axis=-1,name='end_squeeze')
    
    ans_type      = TDense(5,name='ans_type')(pooled_output)
    return tf.keras.Model([input_ for input_ in [unique_id,input_ids,input_mask,segment_ids] 
                           if input_ is not None],
                          [unique_id,start_logits,end_logits,ans_type],
                          name='bert-baseline')    

### Create Model

In [0]:
print("\nGPU Memory\n")
!nvidia-smi --query-gpu=utilization.memory,memory.total,memory.free,memory.used --format=csv


GPU Memory

utilization.memory [%], memory.total [MiB], memory.free [MiB], memory.used [MiB]
0 %, 16280 MiB, 16280 MiB, 0 MiB


In [0]:
## grab bert config
with open(f"{datadir}/bert-joint-baseline/bert_config.json", 'r') as f:
    bert_config = json.load(f)

In [0]:
model= mk_model(bert_config)
print(json.dumps(bert_config, indent=4))

model.summary()
print("\nGPU Memory\n")
!nvidia-smi --query-gpu=utilization.memory,memory.total,memory.free,memory.used --format=csv

{
    "attention_probs_dropout_prob": 0.1,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 1024,
    "initializer_range": 0.02,
    "intermediate_size": 4096,
    "max_position_embeddings": 512,
    "num_attention_heads": 16,
    "num_hidden_layers": 24,
    "type_vocab_size": 2,
    "vocab_size": 30522
}
Model: "bert-baseline"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]  

### Checkpoint

In [0]:
cpkt = tf.train.Checkpoint(model=model)
cpkt.restore(f"{datadir}/bert-joint-baseline/model_cpkt-1").assert_consumed()

### Setting the Flags

In [0]:
#                                   ### This cell was in my copy but not in former
class DummyObject:
    def __init__(self,**kwargs):
        self.__dict__.update(kwargs)

FLAGS=DummyObject(skip_nested_contexts=True,
                 max_position=50,
                 max_contexts=48,
                 max_query_length=64,
                 max_seq_length=512,
                 doc_stride=128,
                 include_unknowns=-1.0,
                 n_best_size=20,
                 max_answer_length=30)

In [0]:
import tqdm
eval_records = f"{datadir}/bert-joint-baseline/nq-test.tfrecords"
if kernel == 'Kaggle' and private_dataset:
    eval_records='nq-test.tfrecords'
if not Path(eval_records).exists():
    # tf2baseline.FLAGS.max_seq_length = 512
    eval_writer = bert_utils.FeatureWriter(
        filename=os.path.join(eval_records),
        is_training=False)

    tokenizer = tokenization.FullTokenizer(vocab_file=f"{datadir}/bert-joint-baseline/vocab-nq.txt", 
                                           do_lower_case=True)

    features = []
    convert = bert_utils.ConvertExamples2Features(tokenizer=tokenizer,
                                                   is_training=False,
                                                   output_fn=eval_writer.process_feature,
                                                   collect_stat=False)

    n_examples = 0
    tqdm_notebook= tqdm.tqdm_notebook if not kernel == 'Kaggle' else None
    for examples in bert_utils.nq_examples_iter(input_file=f"{test_file}", 
                                           is_training=False,
                                           tqdm=tqdm_notebook):
        for example in examples:
            n_examples += convert(example)

    eval_writer.close()
    print('number of test examples: %d, written to file: %d' % (n_examples,eval_writer.num_features))

In [0]:
seq_length = FLAGS.max_seq_length       # bertconfig['max_position_embeddings']
name_to_features = {
      "unique_id": tf.io.FixedLenFeature([], tf.int64),
      "input_ids": tf.io.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.io.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.io.FixedLenFeature([seq_length], tf.int64),
  }

def _decode_record(record, name_to_features=name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.io.parse_single_example(serialized=record, features=name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
        t = example[name]
        if name != 'unique_id': #t.dtype == tf.int64:
            t = tf.cast(t, dtype=tf.int32)
        example[name] = t

    return example

def _decode_tokens(record):
    return tf.io.parse_single_example(serialized=record, 
                                      features={
                                          "unique_id": tf.io.FixedLenFeature([], tf.int64),
                                          "token_map" :  tf.io.FixedLenFeature([seq_length], tf.int64)
                                      })
      


In [0]:
raw_ds = tf.data.TFRecordDataset(eval_records)
token_map_ds = raw_ds.map(_decode_tokens)
decoded_ds = raw_ds.map(_decode_record)
ds = decoded_ds.batch(batch_size=16,drop_remainder=False)

In [0]:
#                                       ### This cell was not in my copy but former
# next(iter(decoded_ds))

In [0]:
result=model.predict_generator(ds, verbose = 1 if verbose else 0)


Instructions for updating:
Please use Model.predict, which supports generators.
    567/Unknown - 632s 1s/step

In [0]:
np.savez_compressed('bert-joint-baseline-output.npz',
                    **dict(zip(['uniqe_id','start_logits','end_logits','answer_type_logits'],
                               result)))

In [0]:
Span = collections.namedtuple("Span", ["start_token_idx", "end_token_idx"])

In [0]:
class ScoreSummary(object):
  def __init__(self):
    self.predicted_label = None
    self.short_span_score = None
    self.cls_token_score = None
    self.answer_type_logits = None

In [0]:
class EvalExample(object):
  """Eval data available for a single example."""
  def __init__(self, example_id, candidates):
    self.example_id = example_id
    self.candidates = candidates
    self.results = {}
    self.features = {}

In [0]:
def get_best_indexes(logits, n_best_size):
  """Get the n-best logits from a list."""
  index_and_score = sorted(
      enumerate(logits[1:], 1), key=lambda x: x[1], reverse=True)
  best_indexes = []
  for i in range(len(index_and_score)):
    if i >= n_best_size:
      break
    best_indexes.append(index_and_score[i][0])
  return best_indexes

def top_k_indices(logits,n_best_size,token_map):
    indices = np.argsort(logits[1:])+1
    indices = indices[token_map[indices]!=-1]
    return indices[-n_best_size:]

### Understanding the code
In the item "answer_type", in the last lines of this block, it is responsible for storing the identified response type, which, according to [github project repository](https://github.com/google-research/language/blob/master/language/question_answering/bert_joint/run_nq.py) can be:
1. UNKNOWN = 0
2. YES = 1
3. NO = 2
4. SHORT = 3
5. LONG = 4

In [0]:
def compute_predictions(example):
  """Converts an example into an NQEval object for evaluation."""
  predictions = []
  n_best_size = FLAGS.n_best_size
  max_answer_length = FLAGS.max_answer_length
  i = 0
  for unique_id, result in example.results.items():
    if unique_id not in example.features:
      raise ValueError("No feature found with unique_id:", unique_id)
    token_map = np.array(example.features[unique_id]["token_map"]) #.int64_list.value
    start_indexes = top_k_indices(result.start_logits,n_best_size,token_map)
    if len(start_indexes)==0:
        continue
    end_indexes   = top_k_indices(result.end_logits,n_best_size,token_map)
    if len(end_indexes)==0:
        continue
    indexes = np.array(list(np.broadcast(start_indexes[None],end_indexes[:,None])))  
    indexes = indexes[(indexes[:,0]<indexes[:,1])*(indexes[:,1]-indexes[:,0]<max_answer_length)]
    for start_index,end_index in indexes:
        summary = ScoreSummary()
        summary.short_span_score = (
            result.start_logits[start_index] +
            result.end_logits[end_index])
        summary.cls_token_score = (
            result.start_logits[0] + result.end_logits[0])
        summary.answer_type_logits = result.answer_type_logits-result.answer_type_logits.mean()
        start_span = token_map[start_index]
        end_span = token_map[end_index] + 1

        # Span logits minus the cls logits seems to be close to the best.
        score = summary.short_span_score - summary.cls_token_score
        predictions.append((score, i, summary, start_span, end_span))
        i += 1 # to break ties

  # Default empty prediction.
  score = -10000.0
  short_span = Span(-1, -1)
  long_span  = Span(-1, -1)
  summary    = ScoreSummary()

  if predictions:
    score, _, summary, start_span, end_span = sorted(predictions, reverse=True)[0]
    short_span = Span(start_span, end_span)
    for c in example.candidates:
      start = short_span.start_token_idx
      end = short_span.end_token_idx
      ## print(c['top_level'],c['start_token'],start,c['end_token'],end)
      if c["top_level"] and c["start_token"] <= start and c["end_token"] >= end:
        long_span = Span(c["start_token"], c["end_token"])
        break

  summary.predicted_label = {
      "example_id": int(example.example_id),
      "long_answer": {
          "start_token": int(long_span.start_token_idx),
          "end_token": int(long_span.end_token_idx),
          "start_byte": -1,
          "end_byte": -1
      },
      "long_answer_score": float(score),
      "short_answers": [{
          "start_token": int(short_span.start_token_idx),
          "end_token": int(short_span.end_token_idx),
          "start_byte": -1,
          "end_byte": -1
      }],
      "short_answer_score": float(score),
      "yes_no_answer": "NONE",
      "answer_type_logits": summary.answer_type_logits.tolist(),
      # here:
      "answer_type": int(np.argmax(summary.answer_type_logits))
  }

  return summary

In [0]:
def compute_pred_dict(candidates_dict, dev_features, raw_results,tqdm=None):
    """Computes official answer key from raw logits."""
    raw_results_by_id = [(int(res.unique_id),1, res) for res in raw_results]

    examples_by_id = [(int(k),0,v) for k, v in candidates_dict.items()]
  
    features_by_id = [(int(d['unique_id']),2,d) for d in dev_features] 
  
    # Join examples with features and raw results.
    examples = []
    print('merging examples...')
    merged = sorted(examples_by_id + raw_results_by_id + features_by_id)
    print('done.')
    for idx, type_, datum in merged:
        if type_==0: #isinstance(datum, list):
            examples.append(EvalExample(idx, datum))
        elif type_==2: #"token_map" in datum:
            examples[-1].features[idx] = datum
        else:
            examples[-1].results[idx] = datum

    # Construct prediction objects.
    print('Computing predictions...')
   
    nq_pred_dict = {}
    #summary_dict = {}
    if tqdm is not None:
        examples = tqdm(examples)
    for e in examples:
        summary = compute_predictions(e)
        #summary_dict[e.example_id] = summary
        nq_pred_dict[e.example_id] = summary.predicted_label

    return nq_pred_dict


In [0]:
def read_candidates_from_one_split(input_path):
  """Read candidates from a single jsonl file."""
  candidates_dict = {}
  print("Reading examples from: %s" % input_path)
  if input_path.endswith(".gz"):
    with gzip.GzipFile(fileobj=tf.io.gfile.GFile(input_path, "rb")) as input_file:
      for index, line in enumerate(input_file):
        e = json.loads(line)
        candidates_dict[e["example_id"]] = e["long_answer_candidates"]
        
  else:
    with tf.io.gfile.GFile(input_path, "r") as input_file:
      for index, line in enumerate(input_file):
        e = json.loads(line)
        candidates_dict[e["example_id"]] = e["long_answer_candidates"]
        # candidates_dict['question'] = e['question_text']
  return candidates_dict

In [0]:
def read_candidates(input_pattern):
  """Read candidates with real multiple processes."""
  input_paths = tf.io.gfile.glob(input_pattern)
  final_dict = {}
  for input_path in input_paths:
    final_dict.update(read_candidates_from_one_split(input_path))
  return final_dict

In [0]:
all_results = [bert_utils.RawResult(*x) for x in zip(*result)]
    
print ("Going to candidates file")

candidates_dict = read_candidates(f"{test_file}")

print ("setting up eval features")

eval_features = list(token_map_ds)

print ("compute_pred_dict")

tqdm_notebook= tqdm.tqdm_notebook
nq_pred_dict = compute_pred_dict(candidates_dict, 
                                       eval_features,
                                       all_results,
                                      tqdm=tqdm_notebook)

predictions_json = {"predictions": list(nq_pred_dict.values())}

print ("writing json")

with tf.io.gfile.GFile(f"{outdir}/predictions.json", "w") as f:
    json.dump(predictions_json, f, indent=4)

Going to candidates file
Reading examples from: /content/data/tensorflow2-question-answering/simplified-nq-test.jsonl
setting up eval features
compute_pred_dict
merging examples...
done.
Computing predictions...



writing json


### Main Change
mmmarchetti had the following note about this notebook.<br>
Here is the small, but main change [to prokaj's notebook]: we created an if to check the predicted response type and thus filter / identify the responses that are passed to the submission file.

#### Filtering the Answers

In [0]:
def create_short_answer(entry):
    answer = []    
    if entry['answer_type'] == 0:
        return ""
    
    elif entry['answer_type'] == 1:
        return 'YES'
    
    elif entry['answer_type'] == 2:
        return 'NO'
        
    elif entry["short_answer_score"] < 1.5:
        return ""
    
    else:
        for short_answer in entry["short_answers"]:
            if short_answer["start_token"] > -1:
                answer.append(str(short_answer["start_token"]) + ":" + str(short_answer["end_token"]))
    
        return " ".join(answer)

def create_long_answer(entry):
    
    answer = []
    
    if entry['answer_type'] == 0:
        return ''
    
    elif entry["long_answer_score"] < 1.5:
        return ""

    elif entry["long_answer"]["start_token"] > -1:
        answer.append(str(entry["long_answer"]["start_token"]) + ":" + str(entry["long_answer"]["end_token"]))
        return " ".join(answer)

#### Creating a DataFrame

In [0]:
test_answers_df = pd.read_json(f"{outdir}/predictions.json")
for var_name in ['long_answer_score','short_answer_score','answer_type']:
    test_answers_df[var_name] = test_answers_df['predictions'].apply(lambda q: q[var_name])
test_answers_df["long_answer"] = test_answers_df["predictions"].apply(create_long_answer)
test_answers_df["short_answer"] = test_answers_df["predictions"].apply(create_short_answer)
test_answers_df["example_id"] = test_answers_df["predictions"].apply(lambda q: str(q["example_id"]))

long_answers = dict(zip(test_answers_df["example_id"], test_answers_df["long_answer"]))
short_answers = dict(zip(test_answers_df["example_id"], test_answers_df["short_answer"]))

test_answers_df.head()

,predictions,long_answer_score,short_answer_score,answer_type,long_answer,short_answer,example_id
0,"{'example_id': -9198586108074363474, 'long_ans...",7.318449,7.318449,3,2777:4157,3054:3058,-9198586108074363474
1,"{'example_id': -9106782542406690435, 'long_ans...",10.478188,10.478188,3,452:667,561:563,-9106782542406690435
2,"{'example_id': -9076824655968712429, 'long_ans...",5.556304,5.556304,4,347:391,387:390,-9076824655968712429
3,"{'example_id': -9030298026304897796, 'long_ans...",8.951933,8.951933,3,620:727,629:636,-9030298026304897796
4,"{'example_id': -9017879887869944707, 'long_ans...",6.273268,6.273268,3,16:271,87:93,-9017879887869944707


### Generating the Submission File

In [0]:
sample_submission = pd.read_csv(f"{datadir}/{competition}/sample_submission.csv")

long_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_long")].apply(lambda q: long_answers[q["example_id"].replace("_long", "")], axis=1)
short_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_short")].apply(lambda q: short_answers[q["example_id"].replace("_short", "")], axis=1)

sample_submission.loc[sample_submission["example_id"].str.contains("_long"), "PredictionString"] = long_prediction_strings
sample_submission.loc[sample_submission["example_id"].str.contains("_short"), "PredictionString"] = short_prediction_strings


In [0]:
sample_submission.to_csv(f"{outdir}/submission.csv", index=False)

In [0]:
sample_submission.head()

,example_id,PredictionString
0,-1011141123527297803_long,223:277
1,-1011141123527297803_short,224:226
2,-1028916936938579349_long,
3,-1028916936938579349_short,
4,-1055197305756217938_long,221:335


* Yes Answers

In [0]:
yes_answers = sample_submission[sample_submission['PredictionString'] == 'YES']
yes_answers

,example_id,PredictionString
27,-1651666484583736653_short,YES
113,-3461207570097431362_short,YES
303,-8260765274544672220_short,YES
309,-871487000194429353_short,YES
553,5962215690907729115_short,YES


* No Answers


In [0]:
no_answers = sample_submission[sample_submission['PredictionString'] == 'NO']
no_answers

,example_id,PredictionString
469,418890410382116795_short,NO


* Balnk Answers

In [0]:
blank_answers = sample_submission[sample_submission['PredictionString'] == '']
blank_answers.head()

,example_id,PredictionString
2,-1028916936938579349_long,
3,-1028916936938579349_short,
8,-1114334749483663139_long,
9,-1114334749483663139_short,
10,-1152268629614456016_long,


In [0]:
blank_answers.count()

example_id          205
PredictionString    205
dtype: int64

In [0]:
## The following code is from the original prokaj notebook
if False and public_dataset:
    # Stuff for debugging
    print(test_answers_df["long_answer_score"].describe())
    print(np.bincount(test_answers_df['answer_type'].values))
    print(test_answers_df[test_answers_df['answer_type']==0])
    print(test_answers_df.predictions.values[-4])
    print(sample_submission.head())

    class ShowPrediction:
        def __init__(self,jsonl_file):
            self._data = {}
            with open(jsonl_file,'r') as f:
                for line in f.readlines():
                    d = json.loads(line)
                    #print(d.keys())
                    self._data[int(d['example_id'])]={
                        'text': d['document_text'],
                        'question': d['question_text']
                    }
        def __call__(self,prediction,include_full_text=True):
            data = self._data[prediction['example_id']]
            res = {'question': data['question']}
            if include_full_text:
                res['text'] = data['text']
            for type_ in ['long_answer','short_answers']:
                ans = prediction[type_]
                if isinstance(ans,list):
                    ans = ans[0]
                start,end = ans['start_token'],ans['end_token']
                res[type_] = ' '.join(data['text'].split()[start:end])
            return res

    show_pred = ShowPrediction(f"{test_file}")
    # the following produces extensive output
    for pred in test_answers_df.predictions[test_answers_df.answer_type==0]:
        print(json.dumps(show_pred(pred,include_full_text=True),indent=4))

    for pred in np.random.choice(predictions_json['predictions'],10):
        print(json.dumps(show_pred(pred,include_full_text=False),indent=4))


In [0]:
! zdump PST
if verbose:
    ! pwd
    list_files(basedir)
! ls -l {outdir}

PST  Sat Jan  4 00:29:52 2020 PST
/content
content/
    bert-joint-baseline-output.npz
    data/
        compdata.flag
        bertfiles.flag
        bert-joint-baseline/
            vocab-nq.txt
            nq-test.tfrecords
            model_cpkt-1.data-00000-of-00002
            bert_config.json
            model_cpkt-1.index
            model_cpkt-1.data-00001-of-00002
        tensorflow2-question-answering/
            simplified-nq-train.jsonl
            simplified-nq-test.jsonl
            sample_submission.csv
lrwxrwxrwx 1 root root 62 Jan  4 00:07 /content/output -> '/content/gdrive/My Drive/Colab/bertqa/BERTjoint_yes_no/output/'


## -- Submitting Results --

In [0]:
raise ExecutionStop("Don't let run all go beyond this")

ExecutionStop: ignored

In [0]:
%%bash
## View Previous Results you have submitted
#kaggle competitions list
kaggle competitions submissions -c {competition}

In [0]:
## Make Submission
# You may be able to submit to some competitions through the API
! kaggle competitions submit -c {competition} -f $RESULT_CSV  -m 'test kaggle cli 3'

In [0]:
!du -lh /

Verify submission by viewing previous results

# Cells below this need to be deleted before submitting Notebook to competition

End of Project Notebook

In [0]:
# Make sure user does not accedentially execute beyond end
raise ExecutionStop("Stopping execution")

# ====== Please fold this stuff up and ignore =====

### SSH Setup
This is only neeeded if you want to log into the Colab machine. Otherwise fold it up and ignore.<br>
To use it you have to create a login at https://ngrok.com
<Details>Thanks to Imad El Hanafi (https://imadelhanafi.com) for showing me how to do this.<p>
You will need to create a free account at https://ngrok.com/ for the SSH tunnel to work.</Details>

File paths are hard coded here because this may be run before program variables are established.

In [0]:
## if you want to use the Kaggle api from command line you will need a kaggle.json file
from pathlib import Path
if Path('/content/gdrive/My Drive/Colab/kaggle.json').exists() or \
                                    Path('/content/kaggle.json').exists():
    pass    # we found a kaggle.json file
else:
    # Give user opportunity to upload a kaggle.json file
    from google.colab import files
    print('Upload kaggle.json if you want the Kaggle API to be availabel in bash.')
    # The files.upload() command is failing sporatically with:
    #   TypeError: Cannot read property '_uploadFiles' of undefined (just run this cell again)
    ! rm "/content/kaggle.json"  2> /dev/null
    files.upload()

In [0]:
%%bash
## Install sshd; Set to allow login and config
apt-get install -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
mkdir -p /var/run/sshd
echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

# set host key to known value (need to test if exist)
gdown -O "/etc/ssh/ssh_host_rsa_key" --id 17Vp-rLM0kLVsIqxo7GkV3YXibGCJ7WCR
chown 600 "/etc/ssh/ssh_host_rsa_key"    # private key will be ignored if not secure
gdown -O "/etc/ssh/ssh_host_rsa_key.pub" --id 1-5yW1EwMdBN0YlRe7McmwDxzmGyvq-gW
# get script to modify login shell to match env of Notebook
gdown -O "/root/init_shell.sh" --id 1-9s5wuq5TkebgKbFvBYy4EeM8c2Ee0xc

# this script will give fix the login shell so Python will work
if [ -f "/root/init_shell.sh" ]; then
    echo "source /root/init_shell.sh" >> /root/.bashrc
fi

In [0]:
## setup ssh user / pass and start sshd

#Generate a random root password
import random, string
sshpass = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(30))

#Set root password
! echo root:$sshpass | chpasswd

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

In [0]:
%%bash
## Get Ngrok from gdrive or try to download (see: https://ngrok.com/download)
if [ -f "/content/bertqa/colab/ngrok-stable-linux-amd64.zip" ]; then
    cp "/content/bertqa/colab/ngrok-stable-linux-amd64.zip" .
    echo "Using ngrok-stable-linux-amd64.zip from gdrive"
else
    wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
fi
unzip -qq -n ngrok-stable-linux-amd64.zip
rm ngrok-stable-linux-amd64.zip

In [0]:
## Get user to enter auth token from ngrok and start tunnel

# Get token from ngrok for the tunnel
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

#### ==============================<br>||====&nbsp;&nbsp;  SSH Login Credentials &nbsp;&nbsp;====||<br>==============================

In [0]:
#@title
print("username: root")
print("password: ", sshpass)

Get the host name and port number at: https://dashboard.ngrok.com/status

```bash
ssh root@0.tcp.ngrok.io -p [ngrok_port]
Login as: root
Servrer refused our key
root@0.tcp.ngrok.io's password: [see above]

(Colab):/content$
```


Install programs

In [0]:
%%bash
# vim
apt-get install vim > /dev/null
echo "set tabstop     =4" >> ~/.vimrc
echo "set softtabstop =4" >> ~/.vimrc
echo "set shiftwidth  =4" >> ~/.vimrc
echo "set expandtab"      >> ~/.vimrc

# js is a JSON processor
apt-get install js > /dev/null

apt-get install tree > /dev/null


If you need to kill Ngrok run this cell

In [0]:
if False:
    !kill $(ps aux | grep './ngrok' | awk '{print $2}')

## -- Misc Notes --

### Prevent Disconnects
Colab periodically disconnects the browser.<br>
You have to save model checkpoints to Google Drive so you don't lose work<br>
See: https://mc.ai/google-colab-drive-as-persistent-storage-for-long-training-runs/<br>
Something to try...<br>
Ctrl+Shift+i in browser and in console run this code...
```
function KeepAlive(){
    console.log("Maintaining Connection");
    document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(KeepAlive,60000);
```
There have been reports of people having their GPU privileges suspended for letting processes run for over 12 hours. It seems that they may penalize you rather than just cutting you off.

### Monitor GPU
```
# From cli I think to monitor GPU while fiting
$ nvidia-smi dmon
$ nvidia-smi pmon
```

### Code From Elsewhere

In [0]:
!nvidia-smi -i 0 -q -d MEMORY,UTILIZATION,POWER,CLOCK,COMPUTE